# Regenerating Dataframe of Quality 90

In [91]:
import pandas as pd

CAS90_re_lt = []

CAS90 = pd.DataFrame.from_csv("90_CAS.csv")
for i in CAS90.values.tolist() :
    Date_ID = i[2].replace("2015/","").replace("/","")
    CAS90_re_lt.append( i + [i[1] + "-" + Date_ID + "-" + str(i[3])])

df_re90 = pd.DataFrame(CAS90_re_lt)
df_re90 = df_re90.rename(columns={0: 'Class', 1: 'Sample',2:"Date",3: "Peak",4:"RT", 5: "scan_number",6 :"Intensity",\
                        7 : "Area%", 8 : "CAS_ID", 9 : "Quality",10 : "Peak_ID"})

# index number regenerating
df_re90.head()

,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
0,CU,CU3,2015/11/03,12,8.897,2351,133877,0.18,000071-43-2,91.0,CU3-1103-12
1,CU,CU3,2015/11/03,19,10.348,2738,42397,0.06,024587-26-6,97.0,CU3-1103-19
2,CU,CU3,2015/11/03,19,10.348,2738,42397,0.06,024587-27-7,95.0,CU3-1103-19
3,CU,CU3,2015/11/03,19,10.348,2738,42397,0.06,019264-50-7,95.0,CU3-1103-19
4,CU,CU3,2015/11/03,21,10.617,2810,505338,0.67,000108-88-3,94.0,CU3-1103-21


# Make estimate bandwidth for Mean-Shift
## conversion scan number to retention time

In [92]:
ten_sec = 10*(1.0/60)
ten_sec2scan = ten_sec / (df_re90.RT/df_re90.scan_number)
ten_sec2scan_avg = ten_sec2scan.mean()

print "10 second to scan number : ", ten_sec2scan_avg

10 second to scan number :  44.2519870842


# A. MeanShift by CAS-ID

http://chrisalbon.com/python/pandas_index_select_and_filter.html

갯수 몇개 이상인건 어떻게 할까....???

In [93]:
x = []
CAS_group = df_re90.groupby("CAS_ID").groups

print "CAS-Group type : ",type(CAS_group)


for cas in CAS_group :
    x.append([cas,CAS_group[cas]])
"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
test_group_df = pd.DataFrame(x)
test_group_df = test_group_df.rename(columns={0: 'CAS_ID', 1: 'Sample_index'})
test_group_df.head()

CAS-Group type :  <type 'dict'>


,CAS_ID,Sample_index
0,000100-66-3,"[17, 49, 87, 139, 165, 242, 267, 338, 372, 404..."
1,999018-64-2,[218]
2,000100-52-7,"[167, 681, 747, 772, 976, 1096, 1165, 1263, 12..."
3,000495-60-3,[3266]
4,001730-37-6,"[29, 32, 35, 59, 94, 96, 99, 119, 122, 124, 12..."


In [94]:
bandwidth = ten_sec2scan_avg

df_re90.ix[CAS_group["000100-66-3"]].head()
# print i,CAS_group[i]

,Class,Sample,Date,Peak,RT,scan_number,Intensity,Area%,CAS_ID,Quality,Peak_ID
17,CU,CU3,2015/11/03,42,14.722,3905,85027,0.11,000100-66-3,95.0,CU3-1103-42
49,CU,CU3,2015/11/25,31,13.305,3527,52072,0.19,000100-66-3,96.0,CU3-1125-31
87,CU,CU3,2015/11/07,31,14.617,3877,54512,0.16,000100-66-3,96.0,CU3-1107-31
139,CU,CU4,2015/11/26,35,13.297,3525,33200,0.07,000100-66-3,93.0,CU4-1126-35
165,CU,CU4,2015/11/18,35,13.346,3538,110131,0.16,000100-66-3,96.0,CU4-1118-35


In [114]:
df_re90.Peak_ID

0        CU3-1103-12
1        CU3-1103-19
2        CU3-1103-19
3        CU3-1103-19
4        CU3-1103-21
5        CU3-1103-24
6        CU3-1103-24
7        CU3-1103-24
8        CU3-1103-26
9        CU3-1103-26
10       CU3-1103-26
11       CU3-1103-32
12       CU3-1103-33
13       CU3-1103-33
14       CU3-1103-33
15       CU3-1103-36
16       CU3-1103-36
17       CU3-1103-42
18       CU3-1103-50
19       CU3-1103-51
20       CU3-1103-64
21       CU3-1103-64
22       CU3-1103-65
23       CU3-1103-72
24       CU3-1103-78
25       CU3-1103-82
26       CU3-1103-85
27       CU3-1103-86
28       CU3-1103-90
29       CU3-1103-95
            ...     
3634     CD5-1109-31
3635     CD5-1109-31
3636     CD5-1109-33
3637     CD5-1109-33
3638     CD5-1109-39
3639     CD5-1109-68
3640     CD5-1109-70
3641     CD5-1109-70
3642     CD5-1109-75
3643     CD5-1109-80
3644     CD5-1109-80
3645     CD5-1109-80
3646     CD5-1109-81
3647     CD5-1109-82
3648     CD5-1109-82
3649     CD5-1109-85
3650     CD5-

In [95]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth


def MeanScan(CAS_ID, cas_index_list , scan_list , bandwidth ) :
    # Y has index number of data frame
    Y = np.array(zip(cas_index_list,np.zeros(len(cas_index_list))), dtype=np.int)    
    X = np.array(zip(scan_list,np.zeros(len(scan_list))), dtype=np.int)
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(X)
    # label of item
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)
    # Let be check
    result = []
    for k in range(n_clusters_):
        my_members = labels == k
        result.append( [CAS_ID , k , list(Y[my_members, 0]) , list(X[my_members, 0])] )
        # CAS-ID  / sub cluster / dataframe index / real scan value
    return result

## Sub grouping by CAS-ID

In [96]:
pre_group = []
bandwidth = ten_sec2scan_avg

for sub in CAS_group :
    CAS_ID = sub
    cas_index_list = CAS_group[sub] 
    scanValue_list = list( df_re90.ix[ cas_index_list ].scan_number.values )
    L = MeanScan( CAS_ID ,cas_index_list ,scanValue_list ,bandwidth) 
    pre_group = pre_group + L

# Put all list to pre_group

In [112]:
GroupedCAS = []

for sgro in pre_group :
    # CAS-ID  / sub cluster / dataframe index / count() / scan number mean / Quality mean / Quality max
    casID = sgro[0]
    subcl_num = sgro[1]
    indeData = sgro[2]
    ScanAVG = np.mean(sgro[3])
    G_sub = df_re90.ix[indeData].Quality
    GroupedCAS.append( [casID , subcl_num , indeData , len(indeData) , ScanAVG , G_sub.mean() , G_sub.max()] )

"~~~~~~~~~~~~~~~~~~show dataframe ~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
GroupedCAS_Df = pd.DataFrame(GroupedCAS)
GroupedCAS_Df = GroupedCAS_Df.rename(columns={0: 'CAS_ID', 1: 'Sub_Cluster' ,2: "Sample_index" , 3: "Count",\
                                             4: "scanValue_avg" , 5: "QualMean", 6: "QualMax"})
GroupedCAS_Df
# GroupedCAS_Df.head()

,CAS_ID,Sub_Cluster,Sample_index,Count,scanValue_avg,QualMean,QualMax
0,000100-66-3,0,"[49, 139, 468, 529, 603, 652, 680, 734, 851, 9...",24,3526.375000,94.833333,97.0
1,000100-66-3,1,"[87, 242, 338, 372, 631, 780, 887, 1135, 2303,...",12,3875.833333,94.916667,97.0
2,000100-66-3,2,"[165, 404, 1553, 1614, 1823, 1985, 2086, 2905,...",11,3536.181818,95.363636,97.0
3,000100-66-3,3,"[482, 1391, 1719, 1909, 2018, 2189, 2382, 2458]",8,3865.000000,95.125000,97.0
4,000100-66-3,4,"[17, 441, 575, 698, 825, 999, 1111]",7,3904.142857,95.428571,96.0
5,000100-66-3,5,"[502, 916, 2839, 3591]",4,3881.750000,94.750000,97.0
6,000100-66-3,6,"[1515, 1949, 2064]",3,3894.000000,94.666667,96.0
7,000100-66-3,7,"[1066, 2564]",2,3869.500000,95.500000,96.0
8,000100-66-3,8,[267],1,3560.000000,96.000000,96.0
9,000100-66-3,9,[3557],1,3912.000000,96.000000,96.0


# B. Check Max Quality CAS-ID removal